# Fake news detector

On this notebook we are going to take best model from the __[Final modelling notebook](https://github.com/pipe11/TFM_fake_news_detector/blob/master/models/11_final_notebook_models_explained.ipynb)__, and load it. It resulted that the best model is XGBOOST, but we are considering 2 models:
1. XGBOOST without features extracted from TFIDF vectorizer
2. XGBOOST with features extracted from TFIDF vectorizer

At first, we are going to consider the easy approach with the first model and later on we will try a predictor with the second model.

## INDEX

[1. Functions](#1.-Functions)

[2.Extract news with url](#2.-Extract-news-with-url)

[3. First model predictor](#3.-First-model-predictor)

[4. Second model predictor](#4.-Second-model-predictor)

## 1. Functions

We need a function to extrat features from the New's text content and headline that we are going to classify as fake or real. For this case we are going to use the function `get_news_features()` from the __[Final feature extraction notebook](https://github.com/pipe11/TFM_fake_news_detector/blob/master/feature_extraction/06_Final_notebook_feature_extraction_explained.ipynb)__. 

Also we are using the following function `get_nsyllables()` to extract the syllables from the text:

### `get_nsyllables()`

In [1]:
def get_nsyllables(text):
    from syltippy import syllabize
    
    text = text.replace(r"*URL*", "url")
    text = re.sub(r'\d+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub(r'[^ \nA-Za-z0-9ÁÉÍÓÚÑáéíóúñ/]+', '', text)
    
    n_syllables = len(syllabize(text)[0])
    
    return n_syllables

### `get_news_features()`

We are not including number of words, sentences, quotes, and syllables, only ratios.

In [2]:
%%time

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords  
from nltk import word_tokenize    
from nltk.stem import SnowballStemmer
from nltk import FreqDist
from string import punctuation
import re
import spacy
#need to switch to es_core_news_md pacy model. es_core_news_lg too large for Heroku.
import es_core_news_md
from lexical_diversity import lex_div as ld
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, csr_matrix

def get_news_features(headline, text):
    
    nlp = spacy.load('es_core_news_lg')

    ## headline ##
    headline = re.sub(r"http\S+", "", headline)
    headline = re.sub(r"http", "", headline)
    headline = re.sub(r"@\S+", "", headline)
    headline = re.sub("\n", " ", headline)
    headline = re.sub(r"(?<!\n)\n(?!\n)", " ", headline)
    headline = headline.replace(r"*NUMBER*", "número")
    headline = headline.replace(r"*PHONE*", "número")
    headline = headline.replace(r"*EMAIL*", "email")
    headline = headline.replace(r"*URL*", "url")
    headline_lower = headline.lower()
    doc_h = nlp(headline_lower)

    list_tokens_h = []
    list_tags_h = []

    for sentence_h in doc_h.sents:
        for token in sentence_h:
            list_tokens_h.append(token.text)

    fdist_h = FreqDist(list_tokens_h)
    syllables_h = get_nsyllables(headline)
    words_h = len(list_tokens_h)

    # headline complexity features
    avg_word_size_h = round(sum(len(word) for word in list_tokens_h) / words_h, 2)
    avg_syllables_word_h = round(syllables_h / words_h, 2)
    unique_words_h = round((len(fdist_h.hapaxes()) / words_h) * 100, 2)
    mltd_h = round(ld.mtld(list_tokens_h), 2)
    ttr_h = round(ld.ttr(list_tokens_h) * 100, 2)

    ## text content##     
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"http", "", text)
    text = re.sub("\n", " ", text)
    text = text.replace(r"*NUMBER*", "número")
    text = text.replace(r"*PHONE*", "número")
    text = text.replace(r"*EMAIL*", "email")
    text = text.replace(r"*URL*", "url")

    # to later calculate upper case letters ratio
    alph = list(filter(str.isalpha, text))
    text_lower = text.lower()
    doc = nlp(text_lower)

    list_tokens = []
    list_pos = []
    list_tag = []
    list_entities = []
    sents = 0

    for entity in doc.ents:
        list_entities.append(entity.label_)

    for sentence in doc.sents:
        sents += 1
        for token in sentence:
            list_tokens.append(token.text)
            list_pos.append(token.pos_)
            list_tag.append(token.tag_)

    # Calculate entities, pos, tag, freq, syllables, words and quotes
    entities = len(list_entities)
    n_pos = nltk.Counter(list_pos)
    n_tag = nltk.Counter(list_tag)
    fdist = FreqDist(list_tokens)
    syllables = get_nsyllables(text)
    words = len(list_tokens)
    quotes = n_tag['PUNCT__PunctType=Quot']

    # complexity features
    avg_word_sentence = round(words / sents, 2)
    avg_word_size = round(sum(len(word) for word in list_tokens) / words, 2)
    avg_syllables_word = round(syllables / words, 2)
    unique_words = round((len(fdist.hapaxes()) / words) * 100, 2)
    ttr = round(ld.ttr(list_tokens) * 100, 2)

    # readability spanish test
    huerta_score = round(206.84 - (60 * avg_syllables_word) - (1.02 * avg_word_sentence), 2)
    szigriszt_score = round(206.835 - ((62.3 * syllables) / words) - (words / sents), 2)

    # stylometric features
    mltd = round(ld.mtld(list_tokens), 2)
    upper_case_ratio = round(sum(map(str.isupper, alph)) / len(alph) * 100, 2)
    entity_ratio = round((entities / words) * 100, 2)
    quotes_ratio = round((quotes / words) * 100, 2)
    propn_ratio = round((n_pos['PROPN'] / words) * 100 , 2)
    noun_ratio = round((n_pos['NOUN'] / words) * 100, 2) 
    pron_ratio = round((n_pos['PRON'] / words) * 100, 2)
    adp_ratio = round((n_pos['ADP'] / words) * 100, 2)
    det_ratio = round((n_pos['DET'] / words) * 100, 2)
    punct_ratio = round((n_pos['PUNCT'] / words) * 100, 2)
    verb_ratio = round((n_pos['VERB'] / words) * 100, 2)
    adv_ratio = round((n_pos['ADV'] / words) * 100, 2)
    sym_ratio = round((n_tag['SYM'] / words) * 100, 2)

    # create df_features
    df_features = pd.DataFrame({'text': text_lower, 'headline':headline_lower, 'words_h': words_h, 'word_size_h': [avg_word_size_h],
                                'avg_syllables_word_h': [avg_syllables_word_h],'unique_words_h': [unique_words_h], 
                                'ttr_h': ttr_h, 'mltd_h': [mltd_h], 'sents': sents, 'words': words,
                                'avg_words_sent': [avg_word_sentence], 'avg_word_size': [avg_word_size], 
                                'avg_syllables_word': avg_syllables_word, 'unique_words': [unique_words], 
                                'ttr': [ttr], 'huerta_score': [huerta_score], 'szigriszt_score': [szigriszt_score],
                                'mltd': [mltd], 'upper_case_ratio': [upper_case_ratio], 'entity_ratio': [entity_ratio],
                                'quotes': quotes, 'quotes_ratio': [quotes_ratio], 'propn_ratio': [propn_ratio], 
                                'noun_ratio': [noun_ratio], 'pron_ratio': [pron_ratio], 'adp_ratio': [adp_ratio],
                                'det_ratio': [det_ratio], 'punct_ratio': [punct_ratio], 'verb_ratio': [verb_ratio],
                                'adv_ratio': [adv_ratio], 'sym_ratio': [sym_ratio]})
    
    return df_features

CPU times: user 1.95 s, sys: 1.75 s, total: 3.7 s
Wall time: 2.31 s


### Functions for the TF-IDF transformation

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords  
from nltk import word_tokenize  
from nltk.data import load  
from nltk.stem import SnowballStemmer  
from string import punctuation

df = pd.read_csv('../data/spanish_corpus_features_v6_no_outliers.csv')

####### TFIDF Transformation to text ########

#Stopword list to use
spanish_stopwords = stopwords.words('spanish')

#Spanish stemmer:
stemmer = SnowballStemmer('spanish')

def stem_tokens(tokens, stemmer):  
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

#Punctuation to remove
non_words = list(punctuation)

#Adding spanish punctuation
non_words.extend(['¿', '¡', '‘', '’', '“', '”'])  
non_words.extend(map(str, range(10)))

def tokenize(text):  
    #Remove punctuation
    text = ''.join([c for c in text if c not in non_words])
    #Tokenize
    tokens =  word_tokenize(text)

    #Stem
    try:
        stems = stem_tokens(tokens, stemmer)
    except Exception as e:
        print(e)
        print(text)
        stems = ['']
    return stems

### Check if it works

In [4]:
headline = input('Insert news headline:')
text = input('Insert news content:')

get_news_features(text, headline)

Insert news headline:Más de 2.000 profesores de Madrid tendrán que repetir el test tras dar positivo por Covid en la primera prueba
Insert news content:Entre 2.000 y 2.500 positivos por Covid, esas son las cifras que arrojan los aproximadamente 67.000 test realizados por la Comunidad de Madrid a los profesores de la región durante esta pasada semana. Así lo ha apuntado este domingo el consejero de Educación, Enrique Ossorio, que ha hecho balance de esas pruebas este domingo durante su visita al albergue juvenil de Los Batanes, en la localidad de Rascafría.  "Queríamos que antes de que empezara el curso todos los profesores se hicieran test serológicos para dar seguridad a todas la sociedad", ha remarcado el consejero, que ha afirmado que Madrid "no ha escatimado" en la realización de estas pruebas a las que han acudido el 90% de los docentes de la región. Unas pruebas que se han saldado con un porcentaje de positivos entre los docentes superior al 3,5%.  Ahora esos profesores cuya prue

,words_h,avg_word_size_h,avg_syllables_word,unique_words_h,mltd_h,sents,words,avg_word_sentence,avg_word_size,unique_words,...,quotes_ratio,propn_ratio,noun_ratio,pron_ratio,adp_ratio,det_ratio,punct_ratio,verb_ratio,adv_ratio,sym_ratio
0,592,4.53,1.65,33.28,68.8,1,20,20.0,4.55,90.0,...,0.0,10.0,15.0,0.0,25.0,10.0,0.0,15.0,5.0,0.0


## 2. Extract news with url

### ¿How to extract the headline and New's content directly with and url?

This is possible with __[Newspaper3k: Article scraping & curation](https://github.com/codelucas/newspaper)__ library for python that extracts the headline and the new's content from any newspaper

In [ ]:
!pip3 install newspaper3k

In [60]:
from newspaper import Article

url = input('Insert url: ')

article = Article(url)
article.download()
article.parse()

if article.authors == []:
    print(article.title, '\n\n', article.text)
    
else:
    print(article.authors, '\n\n', article.title, '\n\n', article.text)

Insert url: https://elpais.com/ideas/2020-09-05/educacion-como-reparar-los-destrozos.html
['J. A. Aunión', 'José Manuel Romero'] 

 Educación: cómo reparar los destrozos 

 La epidemia más devastadora de los últimos 100 años ha destapado las insuficiencias del sistema sanitario y las debilidades del sistema educativo en España. Tres meses de confinamiento con los servicios públicos de salud desbordados como nunca (más de 100.000 ingresos hospitalarios en menos de 100 días; más de 45.000 fallecidos en ese tiempo por la Covid-19) y la educación metida en una pausa incierta (8,2 millones de alumnos y 712.000 profesores encerrados en sus casas a mitad de curso) han agrandado la grieta que sufre desde hace tiempo el Estado de bienestar.

Una comisión del Congreso de los Diputados estudió durante dos meses cómo reparar tanto destrozo. Casi todos los expertos defendieron la necesidad de reformar sectores públicos estratégicos, antes golpeados por los recortes de la última crisis económica y a

We can't do anything if we use this library to extract premium news from newspaper, this will take only a short portion of the news. So later on we need to specify this on the app.

Now lets use this library to extract headline and text features:

In [66]:
from newspaper import Article

url = input('Insert url: ')

article = Article(url)
article.download()
article.parse()

df_features = get_news_features(article.text, article.title)

Insert url: https://www.elmundo.es/madrid/2020/09/06/5f54cdb1fdddff10648b461b.html


In [67]:
df_features

,word_size_h,avg_syllables_word_h,unique_words_h,mltd_h,ttr_h,avg_words_sent,avg_word_size,avg_syllables_word,unique_words,ttr,...,quotes_ratio,propn_ratio,noun_ratio,pron_ratio,adp_ratio,det_ratio,punct_ratio,verb_ratio,adv_ratio,sym_ratio
0,4.53,1.86,31.83,63.0,43.25,20.0,4.55,1.65,90.0,95.0,...,0.0,10.0,15.0,0.0,25.0,10.0,0.0,15.0,5.0,0.0


## 3. First model predictor

### First step is to load the model

In [57]:
import pickle

model = pickle.load(open('../predictors/fake_news_predictorv2.pkl', 'rb'))
model

XGBClassifier(alpha=0.1, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.75, eval_metric='auc',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', lambda=1, learning_rate=0.01,
              max_delta_step=0, max_depth=8, min_child_weight=0.5, missing=nan,
              monotone_constraints='()', n_estimators=1200, n_jobs=1, nthread=1,
              num_parallel_tree=1, random_state=43, reg_alpha=0.100000001,
              reg_lambda=1, scale_pos_weight=0.9423264907135874, seed=43,
              subsample=0.4, tree_method='exact', validate_parameters=1, ...)

In [105]:
xgb_model

XGBClassifier(alpha=0.4, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, eval_metric='auc',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', lambda=0.5, learning_rate=0.03,
              max_delta_step=0, max_depth=6, min_child_weight=2, missing=nan,
              monotone_constraints='()', n_estimators=200, n_jobs=1, nthread=1,
              num_parallel_tree=1, random_state=43, reg_alpha=0.400000006,
              reg_lambda=0.5, scale_pos_weight=0.9775967413441956, seed=43,
              subsample=0.5, tree_method='exact', validate_parameters=1, ...)

### Prediction

We are going to use the spanish language at the strings showing the news classification.

In [219]:
from newspaper import Article

url = input('Insert url: ')

article = Article(url)
article.download()
article.parse()

df_features = get_news_features(article.title, article.text)

# df_features = get_news_features(text, headline)

model = pickle.load(open('../predictors/fake_news_predictorv3.pkl', 'rb'))

numeric_features = ['words_h', 'word_size_h', 'avg_syllables_word_h', 'unique_words_h', 'ttr_h', 'mltd_h', 'sents',
                    'words', 'avg_words_sent', 'avg_word_size', 'avg_syllables_word', 'unique_words', 'ttr', 'mltd', 
                    'huerta_score', 'szigriszt_score','upper_case_ratio', 'entity_ratio', 'quotes', 'quotes_ratio',
                    'propn_ratio', 'noun_ratio', 'adp_ratio', 'det_ratio', 'punct_ratio', 'pron_ratio', 'verb_ratio', 
                    'adv_ratio', 'sym_ratio']

X_predict = df_features[numeric_features]

prob_fake = (model.predict_proba(X_predict)[0][0])*100
prob_real = (model.predict_proba(X_predict)[0][1])*100

if prob_fake >= 65:
    print('Esta noticia es falsa.\nCon una probabilidad del %.0f%%' % prob_fake)

elif (65 >= prob_fake >= 35):
    print('Esta noticia es engañosa o una opinión. \nTiene una probabilidad de %.0f%% de ser verdadera' % prob_real)
else:
    print('Esta noticia es verdadera.\nCon una probabilidad del %.0f%%' % prob_real)

Insert url: https://okdiario.com/opinion/ademas-delincuente-eres-desahogado-pablo-iglesias-6106553
Esta noticia es engañosa o una opinión. 
Tiene una probabilidad de 57% de ser verdadera


Now lets move all this to __[streamlit configuration](https://github.com/pipe11/TFM_fake_news_detector/blob/master/predictors/Streamlit_configuration.ipynb)__.

## 4. Second model predictor

We lauch the second model and now we can productivize it as a predictor. The script its similar to the first predictor. Later on streamlist test and lauch the second and final app.

In [4]:
import pickle

model2 = pickle.load(open('../predictors/fake_news_predictorv4.pkl', 'rb'))
model2

XGBClassifier(alpha=0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.2, eval_metric='auc',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', lambda=0.5, learning_rate=0.02,
              max_delta_step=0, max_depth=6, min_child_weight=0.5, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=1, nthread=1,
              num_parallel_tree=1, random_state=43, reg_alpha=0, reg_lambda=0.5,
              scale_pos_weight=0.9543546694648478, seed=43, subsample=0.4,
              tree_method='exact', validate_parameters=1, ...)

In [6]:
from newspaper import Article

url = input('Insert url: ')

article = Article(url)
article.download()
article.parse()

df_features = get_news_features(article.title, article.text)


numeric_features = ['words_h', 'word_size_h', 'avg_syllables_word_h', 'unique_words_h', 'ttr_h', 'mltd_h', 'sents',
                    'words', 'avg_words_sent', 'avg_word_size', 'avg_syllables_word', 'unique_words', 'ttr', 'mltd', 
                    'huerta_score', 'szigriszt_score','upper_case_ratio', 'entity_ratio', 'quotes', 'quotes_ratio',
                    'propn_ratio', 'noun_ratio', 'adp_ratio', 'det_ratio', 'punct_ratio', 'pron_ratio', 'verb_ratio',
                    'adv_ratio', 'sym_ratio']

#TF-IDF vectorization
tfidf_vectorizer = pickle.load(open('../predictors/tfidf_vectorizer.pkl', 'rb'))
text_predict_vectorized = tfidf_vectorizer.transform(df_features['text'])

X_predict = hstack([csr_matrix(df_features[numeric_features].values), text_predict_vectorized[0:]])

prob_fake = (model2.predict_proba(X_predict)[0][0])*100
prob_real = (model2.predict_proba(X_predict)[0][1])*100

if prob_fake >= 65:
    print('Esta noticia es falsa.\nCon una probabilidad del %.0f%%' % prob_fake)

elif (65 >= prob_fake >= 35):
    print('Esta noticia es engañosa o una opinión. \nTiene una probabilidad de %.0f%% de ser verdadera' % prob_real)
else:
    print('Esta noticia es verdadera.\nCon una probabilidad del %.0f%%' % prob_real)

Insert url: https://www.elmundo.es/espana/2020/09/12/5f5bc69221efa0200a8b45a0.html
Esta noticia es verdadera.
Con una probabilidad del 98%
